<a href="https://colab.research.google.com/github/nschultze/CS577Project/blob/main/ViT_additional_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install accelerate -U
!pip install datasets
!pip install transformers

Import Data

In [9]:
val_path = '/content/drive/MyDrive/Wild_fire_dataset/val'
train_path = '/content/drive/MyDrive/Wild_fire_dataset/train'
test_path = '/content/drive/MyDrive/Wild_fire_dataset/test'

In [10]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda



Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/train/nofire :
['52293279724_c5250f6aa9_o.png', '23862743953_9107eb402a_o.png']

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/test/nofire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/val/nofire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/train/fire :
['51299053281_e87ae6ec37_o.png', '52464696672_7224ba08b9_o.png', '52464402647_50615af1d0_o.png', '52465662020_3fb295fc5a_o.png']

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/test/fire :
['28347651877_ce21ed134e_o.png', '52465192596_bdd7f52f1c_o.png']

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/val/fire :
['52292032847_11ca7d4982_o.png', '52214758010_97f080bc0c_o.png', '52295603790_d2d6a165fc_o.png', '52465478514_e0dd0f619c_o.png']

Total of 12 . List of all non-RGB images: ['52293279724_c5250f6aa9_o.png', '23862743953_9107eb402a_o.png', '51299053281_e87ae6ec37_o.png', '52464696672_7224ba08b9_o.png', '52464402647_50615af1d0_o.png', '52465662020_3fb295fc5a_o.png', '28347651877_ce21ed134e_o.png', '52465192596_bdd7f52f1c_o.png', '52292032847_11ca7d4982_o.png', '52214758010_97f080bc0c_o.png', '52295603790_d2d6a165fc_o.png', '52465478514_e0dd0f619c_o.png']



Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/train/fire :
['fire.309.png', 'fire.354.png', 'fire.367.png', 'fire.548.png']

In [11]:
from PIL import Image
import os

### a function to return a list of images not in RGB format to filter them out
    ### to adhere to ViT transforms and expectations of data structure
def check_alpha_channel(image_folder):
    print('\nNon-RGB images for ', image_folder, ":" )
    problematic_images = []
    for filename in os.listdir(image_folder):
        image_path = os.path.join(image_folder, filename)
        img = Image.open(image_path)
        if img.mode != 'RGB':
            problematic_images.append(filename)
    print(problematic_images)
    return problematic_images


folder_paths = ['/content/drive/MyDrive/Wild_fire_dataset/train/nofire', '/content/drive/MyDrive/Wild_fire_dataset/test/nofire',
                '/content/drive/MyDrive/Wild_fire_dataset/val/nofire','/content/drive/MyDrive/Wild_fire_dataset/train/fire',
                '/content/drive/MyDrive/Wild_fire_dataset/test/fire', '/content/drive/MyDrive/Wild_fire_dataset/val/fire']
images_to_remove = []
for f in folder_paths:
  for img in check_alpha_channel(f):
    images_to_remove.append(img)

print('\nTotal of', len(images_to_remove), '. List of all non-RGB images:', images_to_remove)



Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/train/nofire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/test/nofire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/val/nofire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/train/fire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/test/fire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/val/fire :
[]

Total of 0 . List of all non-RGB images: []


In [12]:
from datasets import load_dataset

DATASET_DIR = '/content/drive/MyDrive/Wild_fire_dataset'
dataset = load_dataset(name="wildfire", path=DATASET_DIR, data_files={"train": '/content/drive/MyDrive/Wild_fire_dataset/train/**',
                                                                      "test": '/content/drive/MyDrive/Wild_fire_dataset/test/**',
                                                                    'val': '/content/drive/MyDrive/Wild_fire_dataset/val/**'})

Resolving data files:   0%|          | 0/2632 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/408 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/398 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

ViT model card: https://huggingface.co/google/vit-base-patch16-224

In [13]:
from transformers import ViTImageProcessor

## load the processor to transform images to be read by ViT
model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)
print(processor)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



Define transform function

In [14]:
def transform(example_batch):
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')

    inputs['label'] = example_batch['label']
    return inputs

In [15]:
### transform the dataset so it is in format prepared to be read by ViT
prepared_ds = dataset.with_transform(transform)

Define Collate Function for unpacking data and training

In [16]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

Define evaluation metric

Here we just use accuracy

In [17]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-17-3044fb6e3895>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics_all(p):
    logits, labels = p.predictions, p.label_ids
    predictions = logits.argmax(axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


# Use the best model from tuning on the wildfire images to now train on the additonal images.

We explore using higher resolution images 384x384

In [19]:
model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor_384 = ViTImageProcessor.from_pretrained(model_name_or_path, size = {"height": 384, "width": 384})

print(processor_384)

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 384,
    "width": 384
  }
}



In [23]:
from transformers import ViTForImageClassification


labels = dataset['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
).to(device)

print(model)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [24]:
from transformers import TrainingArguments
import accelerate

training_args = TrainingArguments(
  output_dir="./vit-base-wildfire-384_additional_images",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=50,
  eval_steps=50,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)


In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics_all,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["val"],
    tokenizer=processor,
)

In [26]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.291600,0.412709,0.824121,0.853393,0.824121,0.810793
100,0.367600,0.312459,0.856784,0.866947,0.856784,0.851129
150,0.091800,0.258709,0.919598,0.925008,0.919598,0.917856
200,0.133300,0.191740,0.927136,0.929468,0.927136,0.927575
250,0.072600,0.238450,0.929648,0.929517,0.929648,0.929558
300,0.095700,0.229176,0.934673,0.936418,0.934673,0.933888
350,0.041500,0.255763,0.932161,0.934868,0.932161,0.931161
400,0.007700,0.232388,0.932161,0.932022,0.932161,0.931935
450,0.040000,0.241171,0.934673,0.935024,0.934673,0.934213
500,0.007000,0.217023,0.934673,0.935024,0.934673,0.934213


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb D

***** train metrics *****
  epoch                    =         4.0
  total_flos               = 759806173GF
  train_loss               =      0.1038
  train_runtime            =  1:10:30.37
  train_samples_per_second =       2.489
  train_steps_per_second   =       0.156


In [27]:
metrics = trainer.evaluate(prepared_ds['val'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9523
  eval_f1                 =     0.9523
  eval_loss               =     0.1836
  eval_precision          =     0.9523
  eval_recall             =     0.9523
  eval_runtime            = 0:02:01.59
  eval_samples_per_second =      3.273
  eval_steps_per_second   =      0.411


# Perfomance on test set

In [28]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


***** test metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9632
  eval_f1                 =     0.9633
  eval_loss               =     0.1477
  eval_precision          =     0.9635
  eval_recall             =     0.9632
  eval_runtime            = 0:02:24.22
  eval_samples_per_second =      2.829
  eval_steps_per_second   =      0.354
